In [2]:
%pip install -q ultralytics kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive  # type: ignore
drive.mount('/content/drive', force_remount=True)
print("Google Drive mounted.")


Mounted at /content/drive
Google Drive mounted.


In [4]:
import kagglehub
import os

os.environ['KAGGLE_DATASETS_PATH'] = '/content/datasets'

In [5]:
path = kagglehub.dataset_download("calebstephen/food-images-and-labels-dataset-for-yolov5")

100%|██████████| 39.5M/39.5M [00:03<00:00, 12.5MB/s]

Extracting files...


In [6]:
import shutil
from pathlib import Path

local_path = Path('/content/food-images-and-labels-dataset-for-yolov5')
shutil.copytree(path, local_path)

PosixPath('/content/food-images-and-labels-dataset-for-yolov5')

In [7]:
%pip install -q ultralytics>=8.0.0 kagglehub>=0.3.0 pandas>=1.5.0 torch>=1.8.0 torchvision>=0.9.0 Pillow>=8.0.0 opencv-python>=4.6.0 matplotlib>=3.3.0 scikit-learn>=1.0.0

In [8]:
from pathlib import Path; import shutil, os
root = Path(local_path)  # Use local copy

In [9]:
import torch
import gc
import os
import sys
from pathlib import Path
from ultralytics import YOLO
import time
import yaml

GOOGLE_DRIVE_BASE_PATH = Path("/content/drive/MyDrive/FoodDetectCheckpoints")
PROJECT_NAME = "food_detection_results"
RUN_NAME = "yolo_detection_run_2"

DRIVE_PROJECT_PATH = GOOGLE_DRIVE_BASE_PATH / PROJECT_NAME
FINAL_SAVE_PATH = DRIVE_PROJECT_PATH / RUN_NAME

DATASET_DIR = Path("/content/food-images-and-labels-dataset-for-yolov5")
DATASET_YAML = DATASET_DIR / "data.yaml"

#  check if the YOLO-formatted dataset exists and is valid
def check_detection_dataset():
    dataset_dir = DATASET_DIR
    yaml_file = DATASET_YAML

    if not dataset_dir.exists():
        print(f"Dataset directory '{dataset_dir}' not found.")
        return False

    required_dirs = [
        dataset_dir / "train" / "images",
        dataset_dir / "train" / "labels",
        dataset_dir / "valid" / "images",
        dataset_dir / "valid" / "labels",
    ]

    for dir_path in required_dirs:
        if not dir_path.exists():
            print(f"Missing directory: {dir_path}")
            return False

    if not yaml_file.exists():
        print(f"Missing YAML config: {yaml_file}")
        return False

    def count_files(path):
        return len(list(path.glob("*")))

    train_images = count_files(dataset_dir / "train" / "images")
    val_images = count_files(dataset_dir / "valid" / "images")
    train_labels = count_files(dataset_dir / "train" / "labels")
    val_labels = count_files(dataset_dir / "valid" / "labels")

    if train_images == 0 or val_images == 0:
        print(f"No images found: train={train_images}, val={val_images}")
        return False

    if train_labels == 0 or val_labels == 0:
        print(f"No labels found: train={train_labels}, val={val_labels}")
        return False

    print(f"YOLO dataset found at: {dataset_dir}")
    print(f"Training: {train_images} images, {train_labels} labels")
    print(f"Validation: {val_images} images, {val_labels} labels")

    try:
        with open(yaml_file, 'r') as f:
            config = yaml.safe_load(f)

        num_classes = config.get('nc', 0)
        class_names = config.get('names', [])

        print(f"Categories: {num_classes} classes")
        print(f"Classes: {class_names[:5]}{'...' if len(class_names) > 5 else ''}")

    except yaml.YAMLError as e:
        print(f"Error reading YAML config file: {e}")
        return False

    return True

# check for GPU availability
def check_gpu():
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"GPU available: {gpu_name} ({gpu_memory:.1f} GB)")
        return True
    else:
        print("No GPU available, will use CPU")
        return False

# save checkpoints in google drive
def setup_google_drive_path():
    print(f"\nSetting up Google Drive path")
    try:
        os.makedirs(DRIVE_PROJECT_PATH, exist_ok=True)
        print(f"Checkpoint directory created/verified: {DRIVE_PROJECT_PATH}")
        return True
    except Exception as e:
        print(f"Failed to access or create Google Drive path: {DRIVE_PROJECT_PATH}")
        print(f"Error: {e}")
        return False

# train
def train_detection_model():

    print("Starting YOLO Food Detection Training")
    print("=" * 50)

    if not check_detection_dataset():
        return False

    if not setup_google_drive_path():
        return False

    gpu_available = check_gpu()

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

    print("\nLoading YOLOv8s detection model...")
    model = YOLO("yolov8s.pt")

    config = {
        'data': str(DATASET_YAML.absolute()),
        'imgsz': 640,
        'epochs': 50,
        'batch': 16 if gpu_available else 8,
        'workers': min(os.cpu_count() // 2, 4) if os.cpu_count() else 2,
        'device': 0 if gpu_available else 'cpu',

        'project': str(DRIVE_PROJECT_PATH),
        'name': RUN_NAME,

        'save': True,
        'save_period': 1,
        'cache': True,
        'verbose': True,
        'plots': True,
        'patience': 15,
        'lr0': 0.01,
        'lrf': 0.01,
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 3,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        'box': 7.5,
        'cls': 0.5,
        'dfl': 1.5,
        'pose': 12.0,
        'kobj': 1.0,
        'label_smoothing': 0.0,
        'nbs': 64,
        'hsv_h': 0.015,
        'hsv_s': 0.7,
        'hsv_v': 0.4,
        'degrees': 0.0,
        'translate': 0.1,
        'scale': 0.5,
        'shear': 0.0,
        'perspective': 0.0,
        'flipud': 0.0,
        'fliplr': 0.5,
        'mosaic': 1.0,
        'mixup': 0.0,
        'copy_paste': 0.0,
        'auto_augment': 'randaugment',
        'erasing': 0.4,
        'crop_fraction': 1.0,
    }

    print(f"\nTraining Configuration:")
    print(f"Model: YOLOv8s Detection")
    print(f"Dataset: {config['data']}")
    print(f"Epochs: {config['epochs']}")
    print(f"Batch size: {config['batch']}")
    print(f"Device: {config['device']}")
    print(f"Checkpoints saving to: {FINAL_SAVE_PATH}")

    print(f"\nStarting detection training...")
    start_time = time.time()

    try:
        results = model.train(**config)

        training_time = time.time() - start_time
        print(f"\nTraining completed in {training_time/60:.1f} minutes")

        print(f"\nRunning final validation...")
        val_results = model.val(data=str(DATASET_YAML.absolute()),
                                 imgsz=640,
                                 batch=config['batch'])

        print(f"\nTraining Results:")
        print(f"Best model saved to: {FINAL_SAVE_PATH / 'weights' / 'best.pt'}")
        print(f"Last model saved to: {FINAL_SAVE_PATH / 'weights' / 'last.pt'}")
        print(f"Results plots saved to: {FINAL_SAVE_PATH}/")

        # Show some key metrics
        if hasattr(val_results, 'box'):
            print(f"\nDetection Metrics:")
            print(f"mAP50: {val_results.box.map50:.3f}")
            print(f"mAP50-95: {val_results.box.map:.3f}")
            print(f"Precision: {val_results.box.mp:.3f}")
            print(f"Recall: {val_results.box.mr:.3f}")

        return True

    except Exception as e:
        print(f"\nTraining failed with error: {e}")
        return False

    finally:
        # Cleanup
        del model
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

def main():
    try:
        success = train_detection_model()
        if success:
            print(f"\nDetection training completed successfully")
            print(f"Check '{FINAL_SAVE_PATH}' on Google Drive for results")
            return 0
        else:
            print(f"\nTraining failed")
            return 1
    except KeyboardInterrupt:
        print(f"\nTraining interrupted by user")
        return 1
    except Exception as e:
        print(f"\nUnexpected error: {e}")
        return 1

if __name__ == "__main__":
    sys.exit(main())

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Starting YOLO Food Detection Training
YOLO dataset found at: /content/food-images-and-labels-dataset-for-yolov5
Training: 704 images, 704 labels
Validation: 65 images, 65 labels
Categories: 12 classes
Classes: ['Apple', 'Chapathi', 'Chicken Gravy', 'Fries', 'Idli']...

Setting up Google Drive path
Checkpoint directory created/verified: /content/drive/MyDrive/FoodDetectCheckpoints/food_detection_results
GPU available: NVIDIA A100-SXM4-80GB (79.3 GB)

Loading YOLOv8s detection model...

Training Configuration:
Model: YOLOv8s Detection
Dataset: /content/food-images-and-labels-dataset-for-yolov5/data.yaml
Epochs: 50
Batch size: 16
Device: 0
Checkpoints saving to: /content/drive/MyDriv

SystemExit: 0

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
